### Laboration Statistiska Metoder
_Björn Winterfjord_

Vi börjar med att importera nödvändiga bibliotek, inklusive filen <span style = "color:green">LR.py</span>, där våra funktioner för linjär regression finns. Därefter läser vi in datafilen <span style = "color:green">Small-diameter-flow.csv</span>.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import LR

data_path = "Small-diameter-flow.csv"
df = pd.read_csv(data_path, index_col=0)


Vi låter responsvektorn y vara kolumnen <span style = "color:green">Flow</span>, och vi ska undersöka om det finns något samband mellan denna och de tre variablerna <span style = "color:green">Kinematic</span>, <span style = "color:green">Geometric</span> och <span style = "color:green">Inertial</span>. Vi sparar dessa tre kolumner i vår designmatris <span style = "color:green">X</span>, tillsammans med en kolumn med 1:or.

Vi tar alltså inte med kolumnen <span style = "color:green">Observer</span> i nuläget, men kommer senare att underöka om det finns något observatörsbias i experimentet.

In [2]:
y = df["Flow"]
X = np.column_stack([np.ones(y.shape[0]), df['Kinematic'], df['Geometric'], df['Inertial']])

Nu utför vi (med hjälp av linjär algebra) vår linjära regression genom att anropa funktionen <span style = "color:green">fit</span> från <span style = "color:green">LR.py</span>. Vi söker ett linjärt samband på formen
$$
y=\beta_0 + \beta_1 [Kinematic] + \beta_2 [Geometric] + \beta_3[Inertial]
$$
där $\beta_0$ är interceptet och $\beta_1$, $\beta_2$ och $\beta_3$ är parametrar för de olika x-variablerna. Vi kommer senare att utvärdera rimligheten i denna modell.

De olika $\beta$-värdena lagras i listan _b_.

In [8]:

m = LR.LinearRegression()
b = m.fit(X, y)
print(f"Vi får β0 = {b[0]:.7f}, β1 = {b[1]:.7f}, β2 = {b[2]:.7f}, β3 = {b[3]:.7f}")


Vi får β0 = -2.5597931, β1 = 0.8687152, β2 = 3.6104182, β3 = -0.7536877


Det finns en risk att de olika x-variablerna inte är oberoende av varandra, och vi kommer därför senare att göra en _dependency check_ för att utröna om så är fallet. Men vi ska börja med att utvärdera ovanstående modell. Det första vi gör är att ta fram dimensionstalet _d_ (vi vet redan att det kommer att bli 3), samt storleken på stickprovet, _n_. Dessa värden kommer vi att behöve vid flera tillfällen i vår kommande analys.

In [4]:
d = m.dimension(b)
n = m.sample_size(y)
print(f"d = {d}, n = {n}")

d = 3, n = 198


Nu ska vi beräkna _variansen_ i vårt material, och för att kunna göra det behöver vi först beräkna SSE, _sum of square errors_. 

In [7]:
# X = designmatrisen, y = responsvektorn, b = listan över β-värden
# n = sample size, d = dimensionen, var = variansen
SSE = m.SSE(y, X, b)
var = m.variance(SSE, n, d)
print(f"Variansen är {var:.7f}")


Variansen är 0.0063087


Värdet på variansen säger oss inte så mycket, dels eftersom den är dimensionslös, och dels eftersom vi inte har något liknande experiment att jämföra med. Dessutom är både våra x- och y-värden logaritmerade.

Nu ska vi beräkna standardavvikelsen $S$ (som ju är roten ur variansen) och variansen bland y-värdena, $S_{yy}$. Därefter kan vi beräkna $SSR$, _sum of square residuals_.

In [9]:
S = m.deviation(var)
Syy = m.Syy(n, y)
SSR = m.SSR(Syy, SSE)

print(f"S = {S:.7f}, Syy = {Syy:.7f}, SSR = {SSR:.7f}")


S = 0.0794272, Syy = 425.1441930, SSR = 423.9203080


Värdet på standardavvikelsen har samma dimension som x-variablerna, men eftersom dessa var logaritmerade, så blir även S dimensionslös. Det är dock ett slags mått på hur stora avvikelserna är från medlet, i medeltal.

SSR och Syy säger oss inte heller så mycket i nuläget, utan behövs mest för att räkna fram de viktiga statistikor vi behöver för att avgöra modellens relevans.

Nu ska vi beräkna F-värdet. Ett högt F-värde 

In [15]:
F = m.Fstatistic(SSR, d, var)
print(F)


22398.7658956827


In [16]:
Rsq = m.Rsq(SSR, Syy)
print(Rsq)


0.9971212473210772


In [17]:
# Dependency check

print(m.Pearsonr(X, 1, 2))
print(m.Pearsonr(X, 1, 3))
print(m.Pearsonr(X, 2, 3))

PearsonRResult(statistic=0.8631350761065918, pvalue=4.5604633624399433e-60)
PearsonRResult(statistic=0.9686707504997812, pvalue=1.588545639896567e-120)
PearsonRResult(statistic=0.9183300308547002, pvalue=7.951572627158216e-81)


In [18]:
# Beräkna varians/kovarians-matris
c = m.var_covar(X, var)
print(c)

[[ 0.1786122  -0.01993074 -0.00513088  0.01567684]
 [-0.01993074  0.00237237  0.00043044 -0.00168614]
 [-0.00513088  0.00043044  0.00108079 -0.0008474 ]
 [ 0.01567684 -0.00168614 -0.0008474   0.00154512]]


In [19]:
# beräkna signifikans för de enskilda β-parametrarna
sig=[]
for a in range(d+1):
    sig.append(m.significance(a,b,c,S))
print(sig)
    

[-76.25702816954549, 224.5517087789827, 1382.6640099494732, -241.40224898279556]


In [20]:
for a in range(d+1):
    print(m.relevance(sig, a, n, d))
    

1.369442865878956e-146
2.279977868333894e-236
0.0
1.9192830886558436e-242


In [21]:
# print(stats.t.cdf(0.025, 200))
# print(stats.t.ppf(0.95,200))
print(stats.t.ppf(1-0.05/2,n-d-1))



1.972267532579456


In [22]:
# calculate confidence interval

for a in range(d+1):
    ci=m.confidence_interval(n,d,var,c,a)
    print(f"Konfidensintervallet för β{a} på 95%-nivån är mellan {b[a]+ci} och {b[a]-ci}")
    

Konfidensintervallet för β0 på 95%-nivån är mellan -2.554534655293283 och -2.5650516160478123
Konfidensintervallet för β1 på 95%-nivån är mellan 0.8693212249244441 och 0.8681091588053166
Konfidensintervallet för β2 på 95%-nivån är mellan 3.610827229208975 och 3.610009129744773
Konfidensintervallet för β3 på 95%-nivån är mellan -0.7531986370351633 och -0.754176809399355
